### Início

#### Importação das Libs e criação da sessão spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

import pyspark.sql.functions as f

import pandas as pd
import requests
import zipfile
import os

import findspark
findspark.init()

spark = (
    SparkSession
    .builder
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)


21/09/17 18:55:37 WARN Utils: Your hostname, iMac-de-Sergio-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.222 instead (on interface en0)
21/09/17 18:55:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/17 18:55:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import pyspark.sql.functions as f

# Local Datalake Paths
local_zones = {
    'landing': './data/landing_zone',
    'raw': './data/raw_zone',
    'staging': './data/staging_zone',
    'consumer': './data/consumer_zone',
}

#### Etapa 1 - Download do Dataset de Pensionistas

In [3]:
# Link para download do arquivo de pensionistas
zipUrl = 'http://repositorio.dados.gov.br/segrt/pensionistas/PENSIONISTAS_082021.zip'

### Landing Zone
# Path do diretorio para gravação do arquivo
outPath = f'{local_zones["landing"]}/pensionistas'
os.makedirs(outPath, exist_ok=True)
    
# baixar arquivo direto da fonte
req = requests.get(zipUrl)
zipOut = f'{outPath}/PENSIONISTAS.zip'
print('Saving:', zipOut)
with open(zipOut, 'wb') as f:
    f.write(req.content)
    f.close()


### Raw Zone
outPath = f'{local_zones["raw"]}/pensionistas'
os.makedirs(outPath, exist_ok=True)

# Unzip - Extract all
with zipfile.ZipFile(zipOut, 'r') as zipObj:
   # Extract all
   listOfFileNames = zipObj.namelist()
   # varre a lista de arquivos - file names
   for fileName in listOfFileNames:
       # Apenas .csv
       if fileName.endswith('.csv'):
           print('Unzip:', fileName, ' -> ', outPath)
           zipObj.extract(fileName, path=outPath)

Saving: ./data/landing_zone/pensionistas/PENSIONISTAS.zip
Unzip: PENSIONISTAS_082021.csv  ->  ./data/raw_zone/pensionistas


#### Etapa 02 - Leitura do dataframe da Raw e formatação para Staging

In [4]:
import pyspark.sql.functions as f

csvFile = f'{local_zones["raw"]}/pensionistas/{fileName}'

# Carregando o Dataframe
df_origem = (
    spark.read
    .format('csv')
    .option('sep',';')
    .option('header',True)
    .load(csvFile)
)

# Selecionando as Colunas representativas
cols = ['NOME DO BENEFICIARIO', 
        'NOME DO ORGAO',
        'DATA DE NASCIMENTO',
        'UF DA UPAG DE VINCULACAO',
        'NATUREZA PENSAO',
        'DATA INICIO DO BENEFICIO',
        'DATA FIM DO BENEFICIO',
        'RENDIMENTO LIQUIDO',
        'PAGAMENTO SUSPENSO']

# Renomeando das colunas
df_pensionistas = (
    df_origem
    .select(cols)
    .withColumnRenamed('NOME DO BENEFICIARIO','nome')
    .withColumnRenamed('NOME DO ORGAO','orgao')
    .withColumnRenamed('DATA DE NASCIMENTO','dtnasc')
    .withColumnRenamed('UF DA UPAG DE VINCULACAO','uf')
    .withColumnRenamed('NATUREZA PENSAO','natpensao')
    .withColumnRenamed('DATA INICIO DO BENEFICIO','dtiniben')
    .withColumnRenamed('DATA FIM DO BENEFICIO','dtfimben')
    .withColumnRenamed('RENDIMENTO LIQUIDO','rendLiquido')
    .withColumnRenamed('PAGAMENTO SUSPENSO','pagsuspenso')
)


# Trantando e convertendo as colunas
df_pensionistas = (
    df_pensionistas
    .select('nome', 'orgao', 'dtnasc', 'uf', 'natpensao', 'dtiniben', 'dtfimben', 'rendLiquido', 'pagsuspenso')
    .withColumn('dtnasc', f.to_date( f.col('dtnasc'),'ddMMyyyy') )
    .withColumn('dtiniben', f.to_date( f.col('dtiniben'),'ddMMyyyy') )
    .withColumn('dtfimben', f.to_date( f.col('dtfimben'),'ddMMyyyy') )
    .withColumn('rendLiquido', f.regexp_replace(f.regexp_replace(f.col('rendLiquido'), "\\.", ""), "\\,", "."))
    .withColumn('rendLiquido', f.col('rendLiquido').cast('float') )
    .withColumn('pagsuspenso', f.when(f.col('pagsuspenso') == 'NAO', False)
                            .when(f.col('pagsuspenso') == 'SIM', True)
                            .otherwise(None) )
    .withColumn('limitMax35', f.expr('round(rendLiquido*0.35,2)') )
    .withColumn('limitMax40', f.expr('round(rendLiquido*0.40,2)') )
)


del df_origem

In [ ]:
df_pensionistas.limit(5).toPandas()

In [ ]:
(
    df_pensionistas
    .groupBy('pagsuspenso')
    .agg(f.count('nome'))
    .toPandas()
)

In [ ]:
(
    df_pensionistas
    .groupBy('uf','natpensao','pagsuspenso')
    .agg(f.count('nome'))
    .toPandas()
)

In [6]:
### Gravação na Staging
outPath = f'{local_zones["staging"]}/pensionistas'
df_pensionistas.write.format('parquet').save(outPath, mode='overwrite')